In [1]:
df = spark.read.option("header","true").csv("s3://bia678/bank.csv")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1670483737903_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)

In [3]:
df.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[age: string, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, day: string, month: string, duration: string, campaign: string, pdays: string, previous: string, poutcome: string, deposit: string]

In [4]:
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df2 =  df.withColumn('age',df['age'].cast('int'))
df2 =  df2.withColumn('duration',df2['duration'].cast('int'))
df2 =  df2.withColumn('campaign',df2['campaign'].cast('int'))
df2 =  df2.withColumn('pdays',df2['pdays'].cast('int'))
df2 =  df2.withColumn('previous',df2['previous'].cast('int'))
stages = []
cateColumns = [
    'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome'
]

for categ in cateColumns:
    indexer = StringIndexer(inputCol = categ, outputCol = categ + 'Index')
    encoder = OneHotEncoderEstimator(
        inputCols=[indexer.getOutputCol()], 
        outputCols=[categ + "classVec"]
    )
    stages += [indexer, encoder]
    
label_indexer = StringIndexer(inputCol = 'deposit', outputCol = 'label')
stages += [label_indexer]
numericCols = ['age', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in cateColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_pip = pipelineModel.transform(df2)
df_pip.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)
 |-- jobIndex: double (nullable = false)
 |-- jobclassVec: vector (nullable = true)
 |-- maritalIndex: double (nullable = false)
 |-- maritalclassVec: vector (nullable = true)
 |-- educationIndex: double (nullable = false)
 |-- educationclassVec: vector (nullable = true)
 |-- defaultIndex: double (nullable = false)
 |-- defaultclassVec: vector (nullable = true)
 |-- housingInd

In [8]:
(training,test)=df_pip.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.ml.classification import DecisionTreeClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
model_dt = dt.fit(training)
pred_dt = model_dt.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
rmse_dt = evaluator.evaluate(pred_dt,{evaluator.metricName: "rmse"})
rmse_dt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4916988552946748

In [12]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
param_grid_dt = ParamGridBuilder()\
            .addGrid(dt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=dt,estimatorParamMaps=param_grid_dt, evaluator=evaluator,numFolds=5)
tuned_model_dt = crossvalidate.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
model_dt_best = tuned_model_dt.bestModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_dt_best = model_dt_best.transform(test)
pred_dt_best
rmse_dt = evaluator.evaluate(pred_dt_best,{evaluator.metricName: "rmse"})
rmse_dt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.45793599183399464

In [15]:
roc_evaluator = BinaryClassificationEvaluator()
dtROC = roc_evaluator.evaluate(pred_dt_best, {roc_evaluator.metricName: "areaUnderROC"})
dtROC

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6980700526239559

In [16]:
dtPR = roc_evaluator.evaluate(pred_dt_best, {roc_evaluator.metricName: "areaUnderPR"})
dtPR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7121474522922859

In [17]:
from pyspark.ml.classification import GBTClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
gbt = GBTClassifier(maxIter=5, maxDepth=2)
model_gbt = gbt.fit(training)
pred_gbt = model_gbt.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
rmse_gbt = evaluator.evaluate(pred_gbt,{evaluator.metricName: "rmse"})
rmse_gbt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4925792478623212

In [20]:
param_grid_gbt = ParamGridBuilder()\
            .addGrid(gbt.maxIter, [1,2,3,4,5])\
            .addGrid(gbt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=gbt,estimatorParamMaps=param_grid_gbt, evaluator=evaluator,numFolds=5)
tuned_model_gbt = crossvalidate.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-20:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3499



In [21]:
model_gbt_best = tuned_model_gbt.bestModel
pred_gbt_best = model_gbt_best.transform(test)
rmse_gbt = evaluator.evaluate(pred_gbt_best,{evaluator.metricName: "rmse"})
rmse_gbt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.45222346893752163

In [22]:
gbtROC = roc_evaluator.evaluate(pred_gbt_best, {roc_evaluator.metricName: "areaUnderROC"})
gbtROC

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8742558266547577

In [23]:
gbtPR = roc_evaluator.evaluate(pred_gbt_best, {roc_evaluator.metricName: "areaUnderPR"})
gbtPR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8293456012152072

In [24]:
## Change the dataset size
(training2,test2)=df_pip.randomSplit([0.6, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
model_dt = dt.fit(training2)
pred_dt = model_dt.transform(test2)
rmse_dt = evaluator.evaluate(pred_dt,{evaluator.metricName: "rmse"})
rmse_dt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4774066621308212

In [26]:
dtROC = roc_evaluator.evaluate(pred_dt, {roc_evaluator.metricName: "areaUnderROC"})
dtROC

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7456246522397258

In [27]:
dtPR = roc_evaluator.evaluate(pred_dt, {roc_evaluator.metricName: "areaUnderPR"})
dtPR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.720101045857142

In [28]:
model_gbt = gbt.fit(training2)
pred_gbt = model_gbt.transform(test2)
rmse_gbt = evaluator.evaluate(pred_gbt,{evaluator.metricName: "rmse"})
rmse_gbt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-26:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 4493



0.4779773815959696

In [29]:
gbtPR = roc_evaluator.evaluate(pred_gbt_best, {roc_evaluator.metricName: "areaUnderPR"})
gbtPR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8293456012152072

In [30]:
gbtPR = roc_evaluator.evaluate(pred_gbt_best, {roc_evaluator.metricName: "areaUnderPR"})
gbtPR

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8293456012152072

In [40]:
##compare the run time
import time
start= time.time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
(training,test)=df_pip.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
model_dt = dt.fit(training)
pred_dt = model_dt.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
evaluator = RegressionEvaluator()
param_grid_dt = ParamGridBuilder()\
            .addGrid(dt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=dt,estimatorParamMaps=param_grid_dt, evaluator=evaluator,numFolds=5)
tuned_model_dt = crossvalidate.fit(training)
model_dt_best = tuned_model_dt.bestModel
pred_dt_best = model_dt_best.transform(test)
rmse_dt = evaluator.evaluate(pred_dt_best,{evaluator.metricName: "rmse"})
rmse_dt
end = time.time()
print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<built-in function time> 1670487458.5575294

In [36]:
start2 = time.time
gbt = GBTClassifier(maxIter=5, maxDepth=2)
model_gbt = gbt.fit(training)
pred_gbt = model_gbt.transform(test)
param_grid_gbt = ParamGridBuilder()\
            .addGrid(gbt.maxIter, [1,2,3,4,5])\
            .addGrid(gbt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=gbt,estimatorParamMaps=param_grid_gbt, evaluator=evaluator,numFolds=5)
tuned_model_gbt = crossvalidate.fit(training)
model_gbt_best = tuned_model_gbt.bestModel
pred_gbt_best = model_gbt_best.transform(test)
rmse_gbt = evaluator.evaluate(pred_gbt_best,{evaluator.metricName: "rmse"})
rmse_gbt
end2 = time.time()
print(start2, end2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-36:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 11399



<built-in function time> 1670487997.7872503

In [37]:
(training,test)=df_pip.randomSplit([0.6, 0.6])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
start3= time.time
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
model_dt = dt.fit(training)
pred_dt = model_dt.transform(test)
evaluator = RegressionEvaluator()
param_grid_dt = ParamGridBuilder()\
            .addGrid(dt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=dt,estimatorParamMaps=param_grid_dt, evaluator=evaluator,numFolds=5)
tuned_model_dt = crossvalidate.fit(training)
model_dt_best = tuned_model_dt.bestModel
pred_dt_best = model_dt_best.transform(test)
rmse_dt = evaluator.evaluate(pred_dt_best,{evaluator.metricName: "rmse"})
rmse_dt
end3 = time.time()
print(start3, end3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<built-in function time> 1670488112.8040798

In [39]:
start4 = time.time
gbt = GBTClassifier(maxIter=5, maxDepth=2)
model_gbt = gbt.fit(training)
pred_gbt = model_gbt.transform(test)
param_grid_gbt = ParamGridBuilder()\
            .addGrid(gbt.maxIter, [1,2,3,4,5])\
            .addGrid(gbt.maxDepth, [1,2,3,4,5])\
            .build()
crossvalidate = CrossValidator(estimator=gbt,estimatorParamMaps=param_grid_gbt, evaluator=evaluator,numFolds=5)
tuned_model_gbt = crossvalidate.fit(training)
model_gbt_best = tuned_model_gbt.bestModel
pred_gbt_best = model_gbt_best.transform(test)
rmse_gbt = evaluator.evaluate(pred_gbt_best,{evaluator.metricName: "rmse"})
rmse_gbt
end4 = time.time()
print(start4, end4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-39:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 14298



<built-in function time> 1670488259.5173657